In [1]:
import xarray as xr
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import Tuple
from collections import defaultdict
data_dir = '../data/'
openaq_dir = data_dir + 'openaq/'
merra2_dir = data_dir + 'merra2/'
surface_flux_dir = merra2_dir + 'surface_flux/'

In [2]:
openaq = pd.read_csv(openaq_dir + 'openaq_integrated.csv')
del openaq['local']
openaq['utc'] = openaq['utc'].apply(np.datetime64) - np.timedelta64(1, 's')
openaq = openaq.rename(columns={'value': 'openaq_pm25', 'utc': 'openaq_time'})
openaq = openaq[openaq['openaq_time'] < np.datetime64('2024-09-01T00:00:00')]
openaq = openaq.reset_index(drop=True)
openaq

,openaq_pm25,openaq_time,parameter,units,site_id,site_name,lat,lon
0,31.0,2019-10-28 18:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615
1,44.0,2019-10-28 20:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615
2,27.0,2019-10-28 21:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615
3,20.0,2019-10-28 23:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615
4,15.0,2019-10-29 00:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615
...,...,...,...,...,...,...,...,...
350372,7.0,2024-08-31 19:59:59,pm25,µg/m³,8881,Uzbekistan,41.3672,69.2725
350373,8.0,2024-08-31 20:59:59,pm25,µg/m³,8881,Uzbekistan,41.3672,69.2725
350374,8.0,2024-08-31 21:59:59,pm25,µg/m³,8881,Uzbekistan,41.3672,69.2725
350375,7.0,2024-08-31 22:59:59,pm25,µg/m³,8881,Uzbekistan,41.3672,69.2725


In [3]:
site_location_dict = openaq.groupby('site_name')[['lat', 'lon']].apply(lambda x: (x.iloc[0]['lat'], x.iloc[0]['lon'])).to_dict()
site_location_dict

{'Baghdad Airnow': (33.3128, 44.3615),
 'Baghdad Embassy': (33.298722, 44.395917),
 'Bahrain': (26.204697, 50.570833),
 'Doha': (25.3337, 51.22953),
 'Dubai': (25.092213, 55.168125),
 'Kuwait': (29.292317, 48.047679),
 'Kyrgyzstan': (40.5372, 72.80531),
 'Tajikistan Dushanbe': (38.557671, 68.775917),
 'Tajikistan Embassy': (38.579708, 68.712176),
 'Turkmenistan': (37.941857, 58.387945),
 'Uzbekistan': (41.3672, 69.2725)}

In [4]:
merra2_lon = np.arange(33.125, 80.625, 0.625)
merra2_lat = np.arange(10.5, 44, 0.5)

In [5]:
def find_closest_merra2(loc: Tuple[int, int]):
    lat, lon = loc
    lat_idx = np.abs(merra2_lat - lat).argmin()
    lon_idx = np.abs(merra2_lon - lon).argmin()
    return merra2_lat[lat_idx], merra2_lon[lon_idx], lat_idx, lon_idx

In [6]:
closest_merra2 = {site: find_closest_merra2(loc) for site, loc in site_location_dict.items()}
closest_merra2

{'Baghdad Airnow': (33.5, 44.375, 46, 18),
 'Baghdad Embassy': (33.5, 44.375, 46, 18),
 'Bahrain': (26.0, 50.625, 31, 28),
 'Doha': (25.5, 51.25, 30, 29),
 'Dubai': (25.0, 55.0, 29, 35),
 'Kuwait': (29.5, 48.125, 38, 24),
 'Kyrgyzstan': (40.5, 72.5, 60, 63),
 'Tajikistan Dushanbe': (38.5, 68.75, 56, 57),
 'Tajikistan Embassy': (38.5, 68.75, 56, 57),
 'Turkmenistan': (38.0, 58.125, 55, 40),
 'Uzbekistan': (41.5, 69.375, 62, 58)}

In [7]:
def find_closest_time_idx(target: np.datetime64, time: np.ndarray):
    return np.abs(time - target).argmin()

# Aerosols

In [ ]:
data_vars = ['DUEXTTAU',
             'SSSMASS25',
             'SSSMASS',
             'OCSMASS',
             'BCSMASS',
             'SSEXTTAU',
             'TOTEXTTAU',
             'BCEXTTAU',
             'SUEXTTAU',
             'OCEXTTAU',
             'SO4SMASS',
             'DUSMASS',
             'DUSMASS25']

In [9]:
# def match_merra2(row: pd.Series, var: str) -> float:
#     site = row['site_name']
#     _, _, lat_idx, lon_idx = closest_merra2[site]
#     date_time = np.datetime64(row['openaq_time'])
#     date = np.datetime_as_string(date_time)
#     date = date[0:4] + date[5:7] + date[8:10]
#     merra2_file = merra2_dir + f'{date}.nc4'
#     merra2 = xr.open_dataset(merra2_file)
#     time_idx = find_closest_time_idx(date_time, merra2['time'].to_numpy())
#     data = merra2[var][time_idx][lat_idx][lon_idx].values.item()
#     merra2.close()
#     return data

In [10]:
# for var in tqdm(data_vars):
#     openaq[var] = openaq.apply(match_merra2, args=(var,), axis=1)

In [11]:
data_vars_values = defaultdict(list)

In [12]:
TIME_IDX = 1
SITE_NAME_IDX = 5

In [13]:
curr_file = None
curr_date = None

In [14]:
for row in tqdm(openaq.to_numpy()):
    site = row[SITE_NAME_IDX]
    _, _, lat_idx, lon_idx = closest_merra2[site]
    date_time = np.datetime64(row[TIME_IDX])
    date = np.datetime_as_string(date_time)
    date = date[0:4] + date[5:7] + date[8:10]
    if curr_date != date:
        curr_date = date
        curr_file = xr.open_dataset(merra2_dir + f'{curr_date}.nc4')
    time_idx = find_closest_time_idx(date_time, curr_file['time'].to_numpy())
    for var in data_vars:
        data_vars_values[var].append(curr_file[var][time_idx][lat_idx][lon_idx].values.item())

100%|██████████| 350377/350377 [1:08:47<00:00, 84.89it/s] 


In [23]:
pd.concat([openaq, pd.DataFrame(data_vars_values)], axis=1).to_csv(data_dir + 'openaq_merra2.csv', index=False)

# Meterology

In [8]:
meterology_vars = ['U2M', 'T500', 'PS', 'Q500', 'T10M', 'Q850', 'V2M', 'V10M', 'T850', 'U10M', 'QV2M', 'QV10M']

In [9]:
meterology_vars_values = defaultdict(list)

In [10]:
TIME_IDX = 1
SITE_NAME_IDX = 5

In [11]:
curr_file = None
curr_date = None

In [12]:
for row in tqdm(openaq.to_numpy()):
    site = row[SITE_NAME_IDX]
    _, _, lat_idx, lon_idx = closest_merra2[site]
    date_time = np.datetime64(row[TIME_IDX])
    date = np.datetime_as_string(date_time)
    date = date[0:4] + date[5:7] + date[8:10]
    if curr_date != date:
        curr_date = date
        curr_file = xr.open_dataset(merra2_dir + f'meterology/{curr_date}.nc4')
    time_idx = find_closest_time_idx(date_time, curr_file['time'].to_numpy())
    for var in meterology_vars:
        meterology_vars_values[var].append(curr_file[var][time_idx][lat_idx][lon_idx].values.item())

100%|██████████| 350377/350377 [51:11<00:00, 114.09it/s] 


In [17]:
df = pd.read_csv(data_dir + 'openaq_merra2.csv')

In [18]:
df.shape

(350377, 21)

In [25]:
pd.concat([df, pd.DataFrame(meterology_vars_values)], axis=1).to_csv(data_dir + 'openaq_merra2_aerosols_metero.csv', index=False)

# Surface Flux

In [8]:
surface_flux_values = defaultdict(list)

In [9]:
TIME_IDX = 1
SITE_NAME_IDX = 5

In [10]:
curr_file = None
curr_date = None

In [11]:
for row in tqdm(openaq.to_numpy()):
    site = row[SITE_NAME_IDX]
    _, _, lat_idx, lon_idx = closest_merra2[site]
    date_time = np.datetime64(row[TIME_IDX])
    date = np.datetime_as_string(date_time)
    date = date[0:4] + date[5:7] + date[8:10]
    if curr_date != date:
        curr_date = date
        curr_file = xr.open_dataset(surface_flux_dir + f'{curr_date}.nc4')
    time_idx = find_closest_time_idx(date_time, curr_file['time'].to_numpy())
    surface_flux_values['PBLH'].append(curr_file['PBLH'][time_idx][lat_idx][lon_idx].values.item())

100%|██████████| 350377/350377 [05:29<00:00, 1062.28it/s]


In [12]:
df = pd.read_csv(data_dir + 'openaq_merra2_aerosols_metero.csv')

In [14]:
df

,openaq_pm25,openaq_time,parameter,units,site_id,site_name,lat,lon,DUEXTTAU,SSSMASS25,...,PS,Q500,T10M,Q850,V2M,V10M,T850,U10M,QV2M,QV10M
0,31.0,2019-10-28 18:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615,0.126073,1.429726e-09,...,100863.953125,0.000599,294.997040,0.007774,0.540911,1.267893,285.016174,-1.380577,0.008615,0.008593
1,44.0,2019-10-28 20:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615,0.122747,1.508852e-09,...,100864.289062,0.000338,293.673523,0.007294,0.649250,1.517674,285.104523,-1.187964,0.008766,0.008760
2,27.0,2019-10-28 21:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615,0.120642,1.512490e-09,...,100850.250000,0.000241,293.124237,0.006970,0.599674,1.396908,285.223206,-1.256243,0.008892,0.008886
3,20.0,2019-10-28 23:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615,0.117869,1.527496e-09,...,100803.468750,0.000234,292.179321,0.006365,0.467283,1.087632,285.518616,-1.540468,0.009197,0.009151
4,15.0,2019-10-29 00:59:59,pm25,µg/m³,8839,Baghdad Airnow,33.3128,44.3615,0.117735,1.501121e-09,...,100778.273438,0.000227,291.771790,0.006159,0.438826,1.023955,285.631927,-1.615325,0.009353,0.009303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350372,7.0,2024-08-31 19:59:59,pm25,µg/m³,8881,Uzbekistan,41.3672,69.2725,0.075855,1.253397e-10,...,93578.359375,0.000185,294.898224,0.004684,-0.385406,-0.840388,292.366333,-1.672649,0.004759,0.004759
350373,8.0,2024-08-31 20:59:59,pm25,µg/m³,8881,Uzbekistan,41.3672,69.2725,0.067616,1.339799e-10,...,93586.679688,0.000234,294.655670,0.004707,-0.354869,-0.739873,291.882599,-1.459667,0.004782,0.004782
350374,8.0,2024-08-31 21:59:59,pm25,µg/m³,8881,Uzbekistan,41.3672,69.2725,0.059099,1.411991e-10,...,93608.781250,0.000281,294.347656,0.004721,-0.439029,-0.874943,291.571838,-1.334082,0.004801,0.004801
350375,7.0,2024-08-31 22:59:59,pm25,µg/m³,8881,Uzbekistan,41.3672,69.2725,0.051028,1.468834e-10,...,93641.179688,0.000300,294.054321,0.004738,-0.557936,-1.089334,291.454865,-1.265444,0.004816,0.004816


In [18]:
pd.concat([df, pd.DataFrame(surface_flux_values)], axis=1).to_csv(data_dir + 'openaq_merra2_all.csv', index=False)